# Dependencies

In [15]:
import nfl_data_py as nfl
import pandas as pd
import sqlite3 as sql
import numpy as np

# Open SQL Connection

In [16]:
cnx = sql.connect("Player_Info.db") #input Database Name
cursor = cnx.cursor() # Creates Cursor if needed comment out

# Import Data

### Data Type information

In [17]:
# Here we are declaring the columns to request from the package NFL_data_py. Then doing some manipulation
# so that the dataframe and subsequent database is cleaner without the need of much user manipulation. 
# These Database will eventually have the following data types:
#
# - player_id         (Type: int64)
# - name              (Type: object) 
# - merge_name        (Type: object) 
# - position          (Type: object) 
# - team              (Type: object) 
# - birthdate         (Type: object) 
# - age               (Type: float64) 
# - draft_year        (Type: float64) 
# - draft_round       (Type: float64) 
# - draft_pick        (Type: float64) 
# - draft_ovr         (Type: float64) 
# - height            (Type: float64) 
# - weight            (Type: float64) 
# - college           (Type: object) 
# - db_season         (Type: int64) 


### Getting Data

In [18]:
import_columns = ['name',
 'merge_name',
 'position',
 'team',
 'birthdate',
 'age',
 'draft_year',
 'draft_round',
 'draft_pick',
 'draft_ovr',
 'height',
 'weight',
 'college',
 'db_season']


id_columns = ['gsis_id',
 'name',
 'merge_name',
 'position',
 'team',
 'birthdate',
 'age',
 'draft_year',
 'draft_round',
 'draft_pick',
 'draft_ovr',
 'height',
 'weight',
 'college',
 'db_season']

#Columns to have within the Databse

In [19]:
ids = nfl.import_ids(import_columns, ['gsis']) 

ids=ids[id_columns]

/usr/local/lib/python3.9/site-packages/nfl_data_py/__init__.py:631: FutureWarning: Passing a set as an indexer is deprecated and will raise in a future version. Use a list instead.
  df = df[set(tgt_ids + columns)]


In [20]:
player_info = ids[ids['gsis_id'].notna()]

In [21]:
# Reindex due to Pandas DataFrame dropping. Could also reindex on player ID, but decided not to.

player_info.reset_index(drop=True, inplace=True) 

In [22]:
player_info.rename(columns ={'gsis_id':'player_id'}, inplace = True)

/var/folders/7x/pqcbs3jx7llff4t8mk23jkmh0000gn/T/ipykernel_28464/213351557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_info.rename(columns ={'gsis_id':'player_id'}, inplace = True)


In [23]:
# Removing the `-` in the player_id column that is added by default from the nfl_data package. Changing type to int which 
# will help us with comparison in later steps. 

player_info['player_id'] = player_info['player_id'].astype(str).apply(lambda x: x.replace('-', '')).astype(int)
player_info.head()

/var/folders/7x/pqcbs3jx7llff4t8mk23jkmh0000gn/T/ipykernel_28464/4175616041.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  player_info['player_id'] = player_info['player_id'].astype(str).apply(lambda x: x.replace('-', '')).astype(int)


,player_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,height,weight,college,db_season
0,36971,Trevor Lawrence,trevor lawrence,QB,JAC,1999-10-06,22.8,2021.0,1.0,1.0,1.0,78.0,220.0,Clemson,2022
1,36945,Justin Fields,justin fields,QB,CHI,1999-03-05,23.4,2021.0,1.0,11.0,11.0,75.0,228.0,Ohio State,2022
2,37013,Zach Wilson,zach wilson,QB,NYJ,1999-08-03,23.0,2021.0,1.0,2.0,2.0,75.0,210.0,Brigham Young,2022
3,37012,Trey Lance,trey lance,QB,SFO,2000-05-12,22.2,2021.0,1.0,3.0,3.0,76.0,226.0,North Dakota State,2022
4,36972,Mac Jones,mac jones,QB,NEP,1998-09-05,23.9,2021.0,1.0,15.0,15.0,75.0,214.0,Alabama,2022


# Output data to an SQL Database

In [24]:
player_info.to_sql('Player_Info', cnx, if_exists='replace', index = False) # Exports data to an SQL Lite Database

3674

# Close SQL Connection

In [25]:
cursor.close()
cnx.close()   

#Closes Connection to DataBase